In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import manifold, decomposition, linear_model, ensemble, neighbors, cross_validation, metrics


import xgboost
from xgboost import DMatrix

/Users/Roman/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv("train_random_oversampling.csv")
test = pd.read_csv("test.csv")

In [3]:
y = train.target
train.drop("target", axis=1, inplace=True)
train.drop("client_id", axis=1, inplace=True)

ans = test[["client_id"]]
test.drop("client_id", axis=1, inplace=True)

In [4]:
train_data, test_data, y_train, y_test = train_test_split(train, y, test_size = 0.4, stratify = y)

In [6]:
Dtrain = DMatrix(data=train_data.to_sparse(), label=y_train)
Dtest = DMatrix(data=test_data.to_sparse(), label=y_test)

watchlist = [(Dtest,'eval'), (Dtrain,'train')]

param = {
    "max_depth" : 6,
    "eta" : 0.1,
    "objective": "binary:logistic",
    "eval_metric": "auc"
}

numround = 500
bst = xgboost.train(params=param, dtrain=Dtrain, num_boost_round=numround, evals=watchlist, early_stopping_rounds=25,\
                   verbose_eval = 25)
pred =  bst.predict(Dtest)
print(metrics.roc_auc_score(y_test, pred))

[0]	eval-auc:0.726783	train-auc:0.728954
Multiple eval metrics have been passed: 'train-auc' will be used for early stopping.

Will train until train-auc hasn't improved in 25 rounds.
[25]	eval-auc:0.766246	train-auc:0.771186
[50]	eval-auc:0.775641	train-auc:0.784171
[75]	eval-auc:0.781106	train-auc:0.791992
[100]	eval-auc:0.7851	train-auc:0.798056
[125]	eval-auc:0.788395	train-auc:0.803333
[150]	eval-auc:0.791357	train-auc:0.808354
[175]	eval-auc:0.794161	train-auc:0.813299
[200]	eval-auc:0.797505	train-auc:0.818788
[225]	eval-auc:0.800128	train-auc:0.823183
[250]	eval-auc:0.80236	train-auc:0.8271
[275]	eval-auc:0.804916	train-auc:0.831433
[300]	eval-auc:0.807469	train-auc:0.835625
[325]	eval-auc:0.809811	train-auc:0.839614
[350]	eval-auc:0.811769	train-auc:0.842945
[375]	eval-auc:0.813612	train-auc:0.846259
[400]	eval-auc:0.815862	train-auc:0.85012
[425]	eval-auc:0.818103	train-auc:0.853964
[450]	eval-auc:0.819863	train-auc:0.856947
[475]	eval-auc:0.822421	train-auc:0.860982
0.824183

In [ ]:
from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [ ]:
def objective(space):
    print(space)
    
    
    
    
    res = metrics.roc_auc_score(y_test, y_ans)
    print(res)
    return{'loss': -res, 'status': STATUS_OK}


space ={
        'n_estimators' : hp.quniform('n_estimators', 100, 600, 1),
        'eta' : hp.uniform('eta', 0.01, 0.4),
        'max_depth' : hp.quniform('max_depth', 3, 10, 1),
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=75,
            trials=trials)

print(best)

In [ ]:
print test.shape

In [7]:
#xgb = xgboost.XGBClassifier(max_depth=8, learning_rate=0.5, n_estimators=515)
#xgb.fit(train.to_sparse(), y)
y_ans = bst.predict(DMatrix(data=test.to_sparse()))

In [8]:
y_ans.shape

(91940,)

In [10]:
ans['val'] = y_ans

/Users/Roman/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [11]:
ans.rename(columns={"client_id":"_ID_", "val":"_VAL_"}, inplace=True)

/Users/Roman/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:2834: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [12]:
ans.shape

(91940, 2)

In [13]:
ans.to_csv("xgboost_random_oversampling_with_weights.csv", index=False, sep=",", header=False)